<a href="https://colab.research.google.com/github/RafalW3bCraft/RWC-FinTunna/blob/main/RWC_FinTunna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RWC-FinTunna



# System_Configuretion

In [ ]:
import torch
import os

print("🚀 SYSTEM INFORMATION")
print("=" * 50)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    print(f"CUDA version: {torch.version.cuda}")
else:
    print("❌ No GPU detected! Please enable GPU in Runtime -> Change runtime type")

os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_DISABLED"] = "true"  # Disable wandb logging initially


In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" -q
!pip install --no-deps "trl<0.9.0" peft accelerate bitsandbytes xformers datasets -q
!pip install transformers diffusers opencv-python pillow matplotlib scikit-learn pandas numpy -q

print("✅ Installation completed!")

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, classification_report
from transformers import (
    AutoTokenizer, AutoModel, AutoModelForCausalLM,
    AutoModelForSequenceClassification, AutoModelForSeq2SeqLM,
    TrainingArguments, Trainer, DataCollatorForSeq2Seq,
    BitsAndBytesConfig, pipeline
)
from datasets import Dataset, load_dataset
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training

# Unsloth for efficient training [5][42][99]
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
from trl import SFTTrainer

# image generation cap
from diffusers import StableDiffusionPipeline, DiffusionPipeline
from PIL import Image

print("🔧 Setting up project configuration...")

In [ ]:
class ProjectConfig:

    STARCODER_MODEL = "bigcode/starcoder2-3b"
    ROBERTA_MODEL = "roberta-base"
    FLAN_T5_MODEL = "google/flan-t5-small"

    # Training parameters
    MAX_SEQ_LENGTH = 1024                             # Reduced for T4 memory
    BATCH_SIZE = 1                                    # Conservative for T4
    GRADIENT_ACCUMULATION_STEPS = 8                   # Simulate larger batch
    LEARNING_RATE = 2e-4                              # Standard for LoRA
    WARMUP_STEPS = 100
    MAX_STEPS = 500                                   # Start with smaller steps

    # LoRA for fine-tuning
    LORA_R = 16
    LORA_ALPHA = 32
    LORA_DROPOUT = 0.05

    # Quantization
    USE_4BIT = True
    BNB_4BIT_COMPUTE_DTYPE = torch.bfloat16
    BNB_4BIT_QUANT_TYPE = "nf4"
    BNB_4BIT_USE_DOUBLE_QUANT = True

    OUTPUT_DIR = "./outputs"
    STARCODER_OUTPUT = f"{OUTPUT_DIR}/starcoder2-3b-finetuned"
    ROBERTA_OUTPUT = f"{OUTPUT_DIR}/roberta-finetuned"
    FLAN_T5_OUTPUT = f"{OUTPUT_DIR}/flan-t5-finetuned"

# Create output dir
import os
for dir_path in [ProjectConfig.OUTPUT_DIR, ProjectConfig.STARCODER_OUTPUT,
                 ProjectConfig.ROBERTA_OUTPUT, ProjectConfig.FLAN_T5_OUTPUT]:
    os.makedirs(dir_path, exist_ok=True)

print(f"  1. Code Generation: {ProjectConfig.STARCODER_MODEL}")
print(f"  2. Text Classification: {ProjectConfig.ROBERTA_MODEL}")
print(f"  3. Text-to-Text: {ProjectConfig.FLAN_T5_MODEL}")


In [ ]:
def print_gpu_utilization():
    if torch.cuda.is_available():
        print(f"🔋 GPU Memory - Allocated: {torch.cuda.memory_allocated()/1024**3:.1f}GB")
        print(f"🔋 GPU Memory - Reserved: {torch.cuda.memory_reserved()/1024**3:.1f}GB")
    else:
        print("❌ No GPU available")

def clear_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("🧹 GPU memory cleared")

def setup_quantization_config():
    return BitsAndBytesConfig(
        load_in_4bit=ProjectConfig.USE_4BIT,
        bnb_4bit_compute_dtype=ProjectConfig.BNB_4BIT_COMPUTE_DTYPE,
        bnb_4bit_quant_type=ProjectConfig.BNB_4BIT_QUANT_TYPE,
        bnb_4bit_use_double_quant=ProjectConfig.BNB_4BIT_USE_DOUBLE_QUANT,
    )

print("🛠️ Utility functions ready!")
print_gpu_utilization()


In [ ]:
from huggingface_hub import notebook_login
try:
    notebook_login()
    print("✅ Ready to proceed (authentication skipped)")
except Exception as e:
    print("failed continuing without authentication...")


In [ ]:
def setup_starcoder2_model():

    clear_memory()

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=ProjectConfig.STARCODER_MODEL,
        max_seq_length=ProjectConfig.MAX_SEQ_LENGTH,
        dtype=None,  # Auto-detect
        load_in_4bit=True,
        # token="hf_..." # Uncomment if using private models
    )

    # Add LoRA fine-tuning
    model = FastLanguageModel.get_peft_model(
        model,
        r=ProjectConfig.LORA_R,
        target_modules=[
            "q_proj", "k_proj", "v_proj", "o_proj",
            "gate_proj", "up_proj", "down_proj"
        ],
        lora_alpha=ProjectConfig.LORA_ALPHA,
        lora_dropout=ProjectConfig.LORA_DROPOUT,
        bias="none",
        use_gradient_checkpointing="unsloth",
        random_state=3407,
        use_rslora=False,
        loftq_config=None,
    )

    print(f"✅ StarCoder2-3B loaded successfully!")
    print_gpu_utilization()

    return model, tokenizer

starcoder_model, starcoder_tokenizer = setup_starcoder2_model()
print("📝 StarCoder2-3B setup function ready!")


In [ ]:
def setup_roberta_model(num_labels=2):

    clear_memory()

    tokenizer = AutoTokenizer.from_pretrained(ProjectConfig.ROBERTA_MODEL)

    quantization_config = setup_quantization_config()

    model = AutoModelForSequenceClassification.from_pretrained(
        ProjectConfig.ROBERTA_MODEL,
        num_labels=num_labels,
        quantization_config=quantization_config,
        device_map="auto"
    )

    model = prepare_model_for_kbit_training(model)

    lora_config = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        r=ProjectConfig.LORA_R,
        lora_alpha=ProjectConfig.LORA_ALPHA,
        target_modules=["query", "value", "key", "dense"],
        lora_dropout=ProjectConfig.LORA_DROPOUT,
        bias="none",
    )

    # Apply LoRA
    model = get_peft_model(model, lora_config)

    print(f"✅ RoBERTa Base loaded successfully!")
    print_gpu_utilization()

    return model, tokenizer

roberta_model, roberta_tokenizer = setup_roberta_model()
print("📝 RoBERTa Base setup function ready!")


In [ ]:
def setup_flan_t5_model():

    clear_memory()

    tokenizer = AutoTokenizer.from_pretrained(ProjectConfig.FLAN_T5_MODEL)

    quantization_config = setup_quantization_config()

    model = AutoModelForSeq2SeqLM.from_pretrained(
        ProjectConfig.FLAN_T5_MODEL,
        quantization_config=quantization_config,
        device_map="auto"
    )

    model = prepare_model_for_kbit_training(model)

    lora_config = LoraConfig(
        task_type=TaskType.SEQ_2_SEQ_LM,
        r=ProjectConfig.LORA_R,
        lora_alpha=ProjectConfig.LORA_ALPHA,
        target_modules=["q", "v", "k", "o", "wi_0", "wi_1", "wo"],
        lora_dropout=ProjectConfig.LORA_DROPOUT,
        bias="none",
    )

    model = get_peft_model(model, lora_config)

    print(f"✅ Flan-T5-Small loaded successfully!")
    print_gpu_utilization()

    return model, tokenizer

flan_t5_model, flan_t5_tokenizer = setup_flan_t5_model()
print("📝 Flan-T5-Small setup function ready!")


In [ ]:
def get_starcoder_training_args():
    return SFTConfig(
        per_device_train_batch_size=ProjectConfig.BATCH_SIZE,
        gradient_accumulation_steps=ProjectConfig.GRADIENT_ACCUMULATION_STEPS,
        warmup_steps=ProjectConfig.WARMUP_STEPS,
        max_steps=ProjectConfig.MAX_STEPS,
        learning_rate=ProjectConfig.LEARNING_RATE,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir=ProjectConfig.STARCODER_OUTPUT,
        max_seq_length=ProjectConfig.MAX_SEQ_LENGTH,
    )

def get_roberta_training_args():
    return TrainingArguments(
        output_dir=ProjectConfig.ROBERTA_OUTPUT,
        learning_rate=ProjectConfig.LEARNING_RATE,
        per_device_train_batch_size=ProjectConfig.BATCH_SIZE,
        per_device_eval_batch_size=ProjectConfig.BATCH_SIZE,
        gradient_accumulation_steps=ProjectConfig.GRADIENT_ACCUMULATION_STEPS,
        num_train_epochs=3,
        weight_decay=0.01,
        evaluation_strategy="steps",
        eval_steps=50,
        save_strategy="steps",
        save_steps=100,
        logging_steps=10,
        warmup_steps=ProjectConfig.WARMUP_STEPS,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        optim="adamw_8bit",
        dataloader_pin_memory=False,
        remove_unused_columns=False,
    )

def get_flan_t5_training_args():
    return TrainingArguments(
        output_dir=ProjectConfig.FLAN_T5_OUTPUT,
        learning_rate=ProjectConfig.LEARNING_RATE,
        per_device_train_batch_size=ProjectConfig.BATCH_SIZE,
        per_device_eval_batch_size=ProjectConfig.BATCH_SIZE,
        gradient_accumulation_steps=ProjectConfig.GRADIENT_ACCUMULATION_STEPS,
        num_train_epochs=3,
        weight_decay=0.01,
        evaluation_strategy="steps",
        eval_steps=50,
        save_strategy="steps",
        save_steps=100,
        logging_steps=10,
        warmup_steps=ProjectConfig.WARMUP_STEPS,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        optim="adamw_8bit",
        predict_with_generate=True,
        dataloader_pin_memory=False,
    )

print("📊 Next steps: Load datasets and start training individual models")

In [ ]:
print(f"\n💾 Output directories created:")
for output_dir in [ProjectConfig.STARCODER_OUTPUT, ProjectConfig.ROBERTA_OUTPUT, ProjectConfig.FLAN_T5_OUTPUT]:
    print(f"  • {output_dir}")

print("\n🔄 Ready to begin Phase One implementation!")
print_gpu_utilization()


# **1️⃣ Code Generation / Programming Datasets (17+)**

| # | Dataset | Description | Notes / Language | Link |
|---|---------|------------|-----------------|------|
| 1 | `codeparrot/codeparrot-clean` | Clean Python GitHub code | Python | [Link](https://huggingface.co/datasets/codeparrot/codeparrot-clean) |
| 2 | `bigcode/the-stack` | Large multi-language code corpus | Multi-language | [Link](https://huggingface.co/datasets/bigcode/the-stack) |
| 3 | `bigcode/the-stack-smol` | Small subset for testing | Multi-language | [Link](https://huggingface.co/datasets/bigcode/the-stack-smol) |
| 4 | `codeparrot/github-jupyter-text` | Jupyter notebooks, code + markdown | Python | [Link](https://huggingface.co/datasets/codeparrot/github-jupyter-text) |
| 5 | `codeparrot/cleaned-java` | Cleaned Java code | Java | [Link](https://huggingface.co/datasets/codeparrot/cleaned-java) |
| 6 | `codeparrot/cleaned-cpp` | Cleaned C++ code | C++ | [Link](https://huggingface.co/datasets/codeparrot/cleaned-cpp) |
| 7 | `codeparrot/cleaned-javascript` | Cleaned JS code | JavaScript | [Link](https://huggingface.co/datasets/codeparrot/cleaned-javascript) |
| 8 | `HuggingFace Code Dataset hub` | Collection of code datasets | Multiple | [Link](https://huggingface.co/datasets?search=code) |
| 9 | `CodeSearchNet` | Code + docstring pairs for search | Python, Java, Go, etc. | [Link](https://huggingface.co/datasets/code_search_net) |
| 10 | `CodeContests` | Competitive programming solutions | Python, C++ | [Link](https://huggingface.co/datasets/codecontests) |
| 11 | `HackerRank` | Code challenges and solutions | Python, Java, C++ | [Link](https://www.kaggle.com/datasets/anishathalye/hackerrank-solutions) |
| 12 | `Funcom` | Code summarization dataset | Python | [Link](https://huggingface.co/datasets/funcom) |
| 13 | `CoNaLa` | Code/Natural Language dataset | Python | [Link](https://huggingface.co/datasets/conala) |
| 14 | `Java-Large` | Large Java dataset | Java | [Link](https://huggingface.co/datasets/mbpp) |
| 15 | `CodeXGLUE` | Multiple code tasks (translation, repair, summarization) | Multi-language | [Link](https://github.com/microsoft/CodeXGLUE) |
| 16 | `MBPP` | Python code + natural language | Python | [Link](https://huggingface.co/datasets/mbpp) |
| 17 | `CodeNet` | Large-scale multi-language code dataset | Multi-language | [Link](https://huggingface.co/datasets/codexglue) |

---

# **2️⃣ NLP / Text Classification / Sentiment Datasets (17+)**

| # | Dataset | Description | Notes | Link |
|---|---------|------------|------|------|
| 1 | `glue/sst2` | Stanford Sentiment Treebank | Binary sentiment | [Link](https://huggingface.co/datasets/glue/viewer/sst2) |
| 2 | IMDB | Movie reviews, binary sentiment | English | [Link](https://huggingface.co/datasets/imdb) |
| 3 | Amazon Reviews | Multi-domain reviews | Multi-class ratings | [Link](https://huggingface.co/datasets/amazon_reviews_multi) |
| 4 | Yelp Reviews | User reviews, multi-class | English | [Link](https://huggingface.co/datasets/yelp_review_full) |
| 5 | Emotion | Multi-class emotion labels | anger, joy, sadness, etc. | [Link](https://huggingface.co/datasets/emotion) |
| 6 | TweetEval | Twitter sentiment | Short informal texts | [Link](https://huggingface.co/datasets/tweeteval) |
| 7 | AG News | News classification | 4 classes | [Link](https://huggingface.co/datasets/ag_news) |
| 8 | DBpedia | Wikipedia text classification | 14 classes | [Link](https://huggingface.co/datasets/dbpedia_14) |
| 9 | 20 Newsgroups | Topic classification | 20 classes | [Link](https://huggingface.co/datasets/20_newsgroups) |
| 10 | CoLA | Grammatical acceptability | Binary | [Link](https://huggingface.co/datasets/glue/viewer/cola) |
| 11 | TREC | Question classification | 6 classes | [Link](https://huggingface.co/datasets/trec) |
| 12 | Financial PhraseBank | Financial sentiment | Positive, negative, neutral | [Link](https://huggingface.co/datasets/financial_phrasebank) |
| 13 | Toxic Comment Classification | Multi-label toxicity | English | [Link](https://huggingface.co/datasets/jigsaw_toxicity_pred) |
| 14 | Multi-Domain Sentiment Dataset (MDSD) | Multi-domain reviews | English | [Link](https://huggingface.co/datasets/mdsd) |
| 15 | Yelp Polarity | Binary sentiment | English | [Link](https://huggingface.co/datasets/yelp_polarity) |
| 16 | Amazon Polarity | Binary sentiment | English | [Link](https://huggingface.co/datasets/amazon_polarity) |
| 17 | SMS Spam Collection | Spam detection | English | [Link](https://huggingface.co/datasets/sms_spam) |

---

# **3️⃣ Instruction-Tuning / Instruction-Following Datasets (17+)**

| # | Dataset | Description | Notes | Link |
|---|---------|------------|------|------|
| 1 | `yahma/alpaca-cleaned` | Instruction + output | 52k examples | [Link](https://huggingface.co/datasets/yahma/alpaca-cleaned) |
| 2 | Stanford Self-Instruct | Multi-domain instructions | 175k+ | [Link](https://huggingface.co/datasets/StanfordAI/self_instruct) |
| 3 | OpenAssistant/oasst1 | Open-source ChatGPT-like instructions | 160k+ | [Link](https://huggingface.co/datasets/OpenAssistant/oasst1) |
| 4 | ShareGPT | Conversations scraped from ChatGPT | Multi-turn dialogue | [Link](https://huggingface.co/datasets/ShareGPT) |
| 5 | Dolly 2.0 | Instruction + output | 15k examples | [Link](https://huggingface.co/datasets/databricks/dolly_2_0) |
| 6 | WizardLM | Instruction + reasoning dataset | 100k+ | [Link](https://huggingface.co/datasets/anon8231489123/wizardlm_unfiltered) |
| 7 | Alpaca-GPT4 | GPT-4 generated instructions | 52k examples | [Link](https://huggingface.co/datasets/tatsu-lab/alpaca_gpt4) |
| 8 | Evol-Instruct | Evolutionary instruction dataset | 100k+ | [Link](https://huggingface.co/datasets/evol-instruct) |
| 9 | FLAN v2 | Google’s FLAN instruction dataset | Multi-task | [Link](https://huggingface.co/datasets/google/flan_v2) |
| 10 | Tulu | Multilingual instruction dataset | 80k+ | [Link](https://huggingface.co/datasets/tulu) |
| 11 | CoT-Instruct | Chain-of-thought instructions | Reasoning tasks | [Link](https://huggingface.co/datasets/cot_instruct) |
| 12 | Super-NaturalInstructions | 1,600+ diverse tasks | Multi-task | [Link](https://huggingface.co/datasets/super_natural_instructions) |
| 13 | MPT-Instruct | MosaicML instruction dataset | Multi-domain | [Link](https://huggingface.co/datasets/mpt_instruct) |
| 14 | Vicuna Dataset | 70k+ instruction-response pairs | Open-source ChatGPT-like | [Link](https://huggingface.co/datasets/vicuna_dataset) |
| 15 | OpenInstruct | 100k+ instructions | Multi-domain | [Link](https://huggingface.co/datasets/openinstruct) |
| 16 | LIMA | Fine-tuning dataset for instruction-following | ~1k high-quality examples | [Link](https://github.com/StanfordAI/LIMA) |
| 17 | Koala | Instruction dataset | ~100k examples | [Link](https://github.com/karpathy/koala) |

---


# Dataset Preprocessing

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

class ProjectConfig:
    STARCODER_MODEL = "bigcode/starcoder2-3b"
    MAX_SEQ_LENGTH = 1024   # adjust per your VRAM (e.g., 512, 2048)
    DATASET_SPLIT = "train[:1%]"  # smaller split for testing

print("Loading dataset...")
code_dataset = load_dataset("codeparrot/codeparrot-clean", split=ProjectConfig.DATASET_SPLIT)

print("Loading tokenizer...")
starcoder_tokenizer = AutoTokenizer.from_pretrained(ProjectConfig.STARCODER_MODEL)

def preprocess_code(example):
    code_sample = example["content"]

    tokenized = starcoder_tokenizer(
        code_sample,
        truncation=True,
        max_length=ProjectConfig.MAX_SEQ_LENGTH,
    )

    return tokenized


print("Tokenizing dataset...")
tokenized_code_dataset = code_dataset.map(
    preprocess_code,
    remove_columns=code_dataset.column_names,
    batched=True,
    num_proc=4
)

print("✅ StarCoder2-3B dataset loaded & tokenized!")
print(tokenized_code_dataset)


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

class ProjectConfig:
    ROBERTA_MODEL = "roberta-base"
    MAX_SEQ_LENGTH = 128

print("Loading tokenizer...")
roberta_tokenizer = AutoTokenizer.from_pretrained(ProjectConfig.ROBERTA_MODEL)

print("Loading GLUE SST-2 dataset...")
glue_train = load_dataset("glue", "sst2", split="train")
glue_val = load_dataset("glue", "sst2", split="validation")

def preprocess_glue(example):
    return roberta_tokenizer(
        example["sentence"],
        truncation=True,
        max_length=ProjectConfig.MAX_SEQ_LENGTH,
        padding="max_length",
    )


print("Tokenizing datasets...")
tokenized_train = glue_train.map(
    preprocess_glue,
    batched=True,
    remove_columns=glue_train.column_names,
    desc="Tokenizing train split"
)

tokenized_val = glue_val.map(
    preprocess_glue,
    batched=True,
    remove_columns=glue_val.column_names,
    desc="Tokenizing validation split"
)

print("✅ RoBERTa Base dataset loaded & tokenized!")
print(tokenized_train)


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

class ProjectConfig:
    FLAN_T5_MODEL = "google/flan-t5-base"
    MAX_SEQ_LENGTH = 256
    DATASET_SPLIT = "train[:5%]"  # small slice for testing

print("Loading tokenizer...")
flan_t5_tokenizer = AutoTokenizer.from_pretrained(ProjectConfig.FLAN_T5_MODEL)

print("Loading Alpaca-cleaned dataset...")
flan_t5_data = load_dataset("yahma/alpaca-cleaned", split=ProjectConfig.DATASET_SPLIT)

def preprocess_alpaca(example):
    input_text = example.get("input", "")
    prompt = example["instruction"]
    if input_text.strip():
        prompt += "\n" + input_text

    target = example["output"]

    model_inputs = flan_t5_tokenizer(
        prompt,
        truncation=True,
        max_length=ProjectConfig.MAX_SEQ_LENGTH,
        padding="max_length",
    )

    labels = flan_t5_tokenizer(
        target,
        truncation=True,
        max_length=ProjectConfig.MAX_SEQ_LENGTH,
        padding="max_length",
    )["input_ids"]

    model_inputs["labels"] = labels
    return model_inputs

print("Tokenizing Alpaca dataset...")
tokenized_alpaca = flan_t5_data.map(
    preprocess_alpaca,
    batched=False,
    remove_columns=flan_t5_data.column_names,
    desc="Tokenizing Alpaca"
)

print("✅ Flan-T5 dataset loaded & tokenized!")
print(tokenized_alpaca[0])


In [ ]:
# Unified # Dataset Preprocessing
from datasets import load_dataset
from transformers import AutoTokenizer

class ProjectConfig:
    # StarCoder2-3B (code)
    STARCODER_MODEL = "bigcode/starcoder2-3b"
    CODE_MAX_SEQ_LENGTH = 1024
    CODE_SPLIT = "train[:1%]"

    # RoBERTa Base (GLUE SST-2)
    ROBERTA_MODEL = "roberta-base"
    ROBERTA_MAX_SEQ_LENGTH = 128

    # Flan-T5 Base (Instruction Tuning)
    FLAN_T5_MODEL = "google/flan-t5-base"
    FLAN_MAX_SEQ_LENGTH = 256
    FLAN_SPLIT = "train[:5%]"

# 1. StarCoder2-3B / CodeParrot
print("\n=== Loading StarCoder2-3B (CodeParrot) ===")
starcoder_tokenizer = AutoTokenizer.from_pretrained(ProjectConfig.STARCODER_MODEL)
try:
    code_dataset = load_dataset("codeparrot/codeparrot-clean", split=ProjectConfig.CODE_SPLIT)
except Exception as e:
    print("Error loading CodeParrot dataset:", e)
    exit(1)

def preprocess_code(example):
    tokenized = starcoder_tokenizer(
        example["content"],
        truncation=True,
        max_length=ProjectConfig.CODE_MAX_SEQ_LENGTH,
    )
    return tokenized

tokenized_code_dataset = code_dataset.map(
    preprocess_code,
    batched=True,
    remove_columns=code_dataset.column_names,
    desc="Tokenizing code dataset"
)
print("✅ StarCoder2-3B dataset loaded & tokenized!")


# 2. GLUE SST-2 / RoBERTa
print("\n=== Loading GLUE SST-2 (RoBERTa) ===")
roberta_tokenizer = AutoTokenizer.from_pretrained(ProjectConfig.ROBERTA_MODEL)

glue_train = load_dataset("glue", "sst2", split="train")
glue_val = load_dataset("glue", "sst2", split="validation")

def preprocess_glue(example):
    result = roberta_tokenizer(
        example["sentence"],
        truncation=True,
        max_length=ProjectConfig.ROBERTA_MAX_SEQ_LENGTH,
        padding="max_length"
    )
    result["labels"] = example["label"]
    return result

tokenized_train = glue_train.map(
    preprocess_glue,
    batched=True,
    remove_columns=glue_train.column_names,
    desc="Tokenizing GLUE train split"
)
tokenized_val = glue_val.map(
    preprocess_glue,
    batched=True,
    remove_columns=glue_val.column_names,
    desc="Tokenizing GLUE validation split"
)
print("✅ GLUE SST-2 dataset loaded & tokenized!")


# 3. Alpaca-Cleaned / Flan-T5
print("\n=== Loading Alpaca-Cleaned (Flan-T5) ===")
flan_t5_tokenizer = AutoTokenizer.from_pretrained(ProjectConfig.FLAN_T5_MODEL)
flan_t5_data = load_dataset("yahma/alpaca-cleaned", split=ProjectConfig.FLAN_SPLIT)

def preprocess_alpaca(example):
    input_text = example.get("input", "")
    prompt = example["instruction"]
    if input_text.strip():
        prompt += "\n" + input_text
    target = example["output"]

    model_inputs = flan_t5_tokenizer(
        prompt,
        truncation=True,
        max_length=ProjectConfig.FLAN_MAX_SEQ_LENGTH,
        padding="max_length"
    )
    labels = flan_t5_tokenizer(
        target,
        truncation=True,
        max_length=ProjectConfig.FLAN_MAX_SEQ_LENGTH,
        padding="max_length"
    )["input_ids"]

    model_inputs["labels"] = labels
    return model_inputs

tokenized_alpaca = flan_t5_data.map(
    preprocess_alpaca,
    batched=False,
    remove_columns=flan_t5_data.column_names,
    desc="Tokenizing Alpaca dataset"
)
print("✅ Alpaca-cleaned dataset loaded & tokenized!")


print("Sample from tokenized code dataset:", tokenized_code_dataset[0])
print("Sample from tokenized GLUE train:", tokenized_train[0])
print("Sample from tokenized Alpaca:", tokenized_alpaca[0])


# This is Training loops

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

class ProjectConfig:
    STARCODER_MODEL = "bigcode/starcoder2-3b"
    STARCODER_OUTPUT = "./starcoder2-finetuned"
    MAX_SEQ_LENGTH = 1024

def setup_starcoder2_model():
    print("Loading StarCoder2-3B model...")
    model = AutoModelForCausalLM.from_pretrained(
        ProjectConfig.STARCODER_MODEL,
        device_map="auto",
        torch_dtype=torch.float16,  # adjust to float32 if GPU memory is tight
    )
    tokenizer = AutoTokenizer.from_pretrained(ProjectConfig.STARCODER_MODEL)
    return model, tokenizer

starcoder_model, starcoder_tokenizer = setup_starcoder2_model()


# Apply LoRA (PEFT)
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # StarCoder2-3B attention layers
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

starcoder_model = get_peft_model(starcoder_model, lora_config)

# Define Training Args
from transformers import TrainingArguments

def get_starcoder_training_args():
    return TrainingArguments(
        output_dir=ProjectConfig.STARCODER_OUTPUT,
        per_device_train_batch_size=1,  # adjust to VRAM
        gradient_accumulation_steps=8,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        save_strategy="steps",
        save_steps=200,
        max_steps=1000,  # small number for testing
        optim="paged_adamw_8bit",
        save_total_limit=2,
        report_to="none",
    )

training_args = get_starcoder_training_args()

# Setup SFTTrainer
starcoder_trainer = SFTTrainer(
    model=starcoder_model,
    tokenizer=starcoder_tokenizer,
    train_dataset=tokenized_code_dataset,
    eval_dataset=None,  # add a validation split for real tasks
    peft_config=lora_config,
    dataset_text_field="input_ids",
    max_seq_length=ProjectConfig.MAX_SEQ_LENGTH,
    args=training_args
)

# Start Training
print("Starting StarCoder2-3B fine-tuning...")
starcoder_trainer.train()

# Save the fine-tuned model
print(f"Saving fine-tuned model to {ProjectConfig.STARCODER_OUTPUT} ...")
starcoder_model.save_pretrained(ProjectConfig.STARCODER_OUTPUT)
starcoder_tokenizer.save_pretrained(ProjectConfig.STARCODER_OUTPUT)
print("✅ Model saved!")


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding

class ProjectConfig:
    ROBERTA_MODEL = "roberta-base"
    ROBERTA_MAX_SEQ_LENGTH = 128
    ROBERTA_OUTPUT = "./roberta-sst2-finetuned"
    TRAIN_BATCH_SIZE = 16
    EVAL_BATCH_SIZE = 16
    LEARNING_RATE = 2e-5
    NUM_EPOCHS = 3


def setup_roberta_model():
    tokenizer = AutoTokenizer.from_pretrained(ProjectConfig.ROBERTA_MODEL)
    model = AutoModelForSequenceClassification.from_pretrained(
        ProjectConfig.ROBERTA_MODEL,
        num_labels=2  # SST-2 is binary classification
    )
    return model, tokenizer

roberta_model, roberta_tokenizer = setup_roberta_model()


def get_roberta_training_args():
    return TrainingArguments(
        output_dir=ProjectConfig.ROBERTA_OUTPUT,
        evaluation_strategy="steps",
        eval_steps=200,
        save_steps=200,
        save_total_limit=2,
        learning_rate=ProjectConfig.LEARNING_RATE,
        per_device_train_batch_size=ProjectConfig.TRAIN_BATCH_SIZE,
        per_device_eval_batch_size=ProjectConfig.EVAL_BATCH_SIZE,
        num_train_epochs=ProjectConfig.NUM_EPOCHS,
        weight_decay=0.01,
        logging_steps=50,
        logging_dir="./logs",
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        fp16=True,
        report_to="none",
    )

training_args = get_roberta_training_args()

data_collator = DataCollatorWithPadding(tokenizer=roberta_tokenizer)

roberta_trainer = Trainer(
    model=roberta_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=roberta_tokenizer,
    data_collator=data_collator
)

print("Starting RoBERTa SST-2 fine-tuning...")
roberta_trainer.train()

print(f"Saving fine-tuned model to {ProjectConfig.ROBERTA_OUTPUT} ...")
roberta_model.save_pretrained(ProjectConfig.ROBERTA_OUTPUT)
roberta_tokenizer.save_pretrained(ProjectConfig.ROBERTA_OUTPUT)
print("✅ RoBERTa SST-2 model saved!")


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForSeq2Seq
import torch

class ProjectConfig:
    FLAN_T5_MODEL = "google/flan-t5-base"
    FLAN_MAX_SEQ_LENGTH = 256
    FLAN_SPLIT = "train[:5%]"
    FLAN_T5_OUTPUT = "./flan-t5-alpaca-finetuned"
    TRAIN_BATCH_SIZE = 4
    EVAL_BATCH_SIZE = 4
    LEARNING_RATE = 5e-5
    NUM_EPOCHS = 3

def setup_flan_t5_model():
    tokenizer = AutoTokenizer.from_pretrained(ProjectConfig.FLAN_T5_MODEL)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ProjectConfig.FLAN_T5_MODEL,
        device_map="auto",
        torch_dtype=torch.float32  # Changed to float32
    )
    return model, tokenizer

flan_t5_model, flan_t5_tokenizer = setup_flan_t5_model()

def get_flan_t5_training_args():
    return TrainingArguments(
        output_dir=ProjectConfig.FLAN_T5_OUTPUT,
        per_device_train_batch_size=ProjectConfig.TRAIN_BATCH_SIZE,
        per_device_eval_batch_size=ProjectConfig.EVAL_BATCH_SIZE,
        num_train_epochs=ProjectConfig.NUM_EPOCHS,
        learning_rate=ProjectConfig.LEARNING_RATE,
        logging_steps=50,
        save_strategy="steps",
        save_steps=200,
        save_total_limit=2,
        fp16=False, # Disabled FP16
        # evaluation_strategy="no", # Removed as no eval dataset is provided
        report_to="none",
    )

training_args = get_flan_t5_training_args()

# Data collator for Seq2Seq
data_collator = DataCollatorForSeq2Seq(
    tokenizer=flan_t5_tokenizer,
    model=flan_t5_model,
    padding=True
)

flan_trainer = Trainer(
    model=flan_t5_model,
    args=training_args,
    train_dataset=tokenized_alpaca,
    eval_dataset=None,
    data_collator=data_collator,
    tokenizer=flan_t5_tokenizer
)

# Train
print("Starting Flan-T5 Alpaca fine-tuning...")
flan_trainer.train()

print(f"Saving fine-tuned Flan-T5 model to {ProjectConfig.FLAN_T5_OUTPUT} ...")
flan_t5_model.save_pretrained(ProjectConfig.FLAN_T5_OUTPUT)
flan_t5_tokenizer.save_pretrained(ProjectConfig.FLAN_T5_OUTPUT)
print("✅ Flan-T5 model saved!")

In [ ]:
# For code, test pass@1 and run a few completions
# For text, use Trainer.evaluate() and print metrics
# Save models to Hugging Face or GDrive